In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import random
from random import randrange
import torch
from torch import nn, optim
from torchvision import datasets, transforms
import torch.nn.functional as F
import pickle

In [2]:
methods_dict = {1:"Missing_Data", 2:"Outliers", 3:"Labeling_Errors", 4:"Feature_Noise"} 
def data_corruption(method, data, labels, corrupt_p):
    corruption_method = methods_dict[method]
    if corrupt_p <= 0 or corrupt_p >= 1:
        print("Please choose a valid value for the corruption parameter (positive, above 0 and less than 1.)")
        return None
    
    
    # Go through each image, and create a randomized mask which sets pixels to a value of 0.
    data_corrupted = []
    if corruption_method == "Missing_Data":
        for image in data:
            mask = np.zeros(image.shape[0], dtype=int)
            mask[int(image.shape[0]*corrupt_p):] = 1
            random.shuffle(mask)
            mask = mask.astype(bool)
            corrupted_image = np.where(mask == False, 0, image)
            data_corrupted.append(corrupted_image)
        
        return data_corrupted
    
    if corruption_method == "Outliers":
        print("not yet implemented")
        
    if corruption_method == "Labeling_Errors":
        if len(labels) > 0:
            label_names = np.unique(labels)
            mask = np.zeros(len(labels), dtype=int)
            mask[int(len(labels)*corrupt_p):] = 1
            random.shuffle(mask)
            mask = mask.astype(bool)
            corrupted_labels = np.where(mask == False, "needs_change", labels)
            for i in range(len(labels)):
                if corrupted_labels[i] == "needs_change":
                    options = np.delete(label_names, np.where(np.unique(labels) == labels[i])) ## Using np.unique to find the set of labels.
                    corrupted_labels[i] = random.choice(options)

            corrupted_labels = corrupted_labels.astype(type(labels[i]))
            return corrupted_labels
        
        
        else:
            print("Please use an actual list for the labels")
        
    if corruption_method == "Feature_Noise":
        print("not yet implemented")
    

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self, model_params):
        super(NeuralNetwork, self).__init__()
        
        input_dim = model_params["input_dim"]
        hidden_dim = model_params["hidden_dim"]
        output_dim = model_params["output_dim"]
        
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        return out

In [4]:
class Net_MNIST(nn.Module):
    def __init__(self, model_params):
        super(Net_MNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
            x = self.pool1(torch.relu(self.conv1(x)))
            x = self.pool2(torch.relu(self.conv2(x)))
            x = x.view(-1, 32 * 4 * 4)
            x = torch.relu(self.fc1(x))
            x = torch.relu(self.fc2(x))
            x = self.fc3(x)
            return x
        
        
class Net_CIFAR10(nn.Module):
    def __init__(self, model_params):
        super(Net_CIFAR10, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
            x = self.pool1(torch.relu(self.conv1(x)))
            x = self.pool2(torch.relu(self.conv2(x)))
            x = x.view(-1, 32 * 5 * 5)
            x = torch.relu(self.fc1(x))
            x = torch.relu(self.fc2(x))
            x = self.fc3(x)
            return x
        


In [5]:
class LogisticRegression(nn.Module):
    def __init__(self, model_params):
        super(LogisticRegression, self).__init__()
        
        input_dim = model_params["input_dim"]
        output_dim = model_params["output_dim"]
        
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out


In [6]:

# Currently only allows for scrambling of labels as the corruption method

def FedAvg(model, data, model_params, training_params, corrupt=False, cp=0.5):
    acc_dict = dict()
    
    model_dict = {1: NeuralNetwork, 2: LogisticRegression, 3: "Net"}
    model = model_dict[model]
    
    num_clients = training_params["num_clients"]
    epochs = training_params["epochs"]
    batch_size = training_params["batch_size"]
    lr = training_params["lr"]
    
    
    if data == "MNIST":
        # Load MNIST dataset - Normalized (MEAN STD)
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
        test_dataset = datasets.MNIST('../data', train=False, download=True, transform=transform)
        if model == "Net":
            model = Net_MNIST
    
    if data == "CIFAR10":
        transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, padding=4),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
        train_dataset = datasets.CIFAR10('../data', train=True, download=True, transform=transform)
        test_dataset = datasets.CIFAR10('../data', train=False, download=True, transform=transform)
        if model == "Net":
            model = Net_CIFAR10

    # Split data among clients
    client_datasets = torch.utils.data.random_split(train_dataset, [len(train_dataset) // num_clients] * num_clients)
    
    # Initialize global model
    central_model = model(model_params)

    # Train global model using federated averaging
    central_optimizer = optim.SGD(central_model.parameters(), lr=lr)
    central_criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        central_model.train()

        # Train local models on each client
        local_models = []
        for client_dataset in client_datasets:
            local_model = model(model_params)
            local_model.load_state_dict(central_model.state_dict())
            local_optimizer = optim.SGD(local_model.parameters(), lr=lr, momentum=0.9)
            local_criterion = nn.CrossEntropyLoss()

            for local_epoch in range(epochs):
                local_model.train()

                for local_data, local_target in torch.utils.data.DataLoader(client_dataset, batch_size=batch_size, shuffle=True):
                    if corrupt:
                        local_target = torch.as_tensor(data_corruption(3, local_data, local_target.tolist(), cp)).type(torch.LongTensor)
                    local_optimizer.zero_grad()
                    if model == Net_MNIST or model == Net_CIFAR10:
                        local_output = local_model(local_data)
                    else:
                        local_output = local_model(local_data.view(local_data.shape[0], -1))
                        
                    local_loss = local_criterion(local_output, local_target)
                    local_loss.backward()
                    local_optimizer.step()

            local_models.append(local_model)

        # Update central model using federated averaging
        for name, param in central_model.named_parameters():
            if name.endswith('.bias'):
                continue

            local_params = torch.stack([local_model.state_dict()[name] for local_model in local_models])
            central_mean = local_params.mean(0)
            param.data = central_mean.data

        central_loss = 0
        central_accuracy = 0
        central_optimizer.zero_grad()

        for central_data, central_target in torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True):
            if model == Net_MNIST or model == Net_CIFAR10:
                central_output = central_model(central_data)
            else:
                central_output = central_model(central_data.view(central_data.shape[0], -1))
            central_loss += central_criterion(central_output, central_target)
            central_accuracy += (central_output.argmax(1) == central_target).float().sum()

        central_loss /= len(test_dataset)
        central_accuracy /= len(test_dataset)

        central_loss.backward()
        central_optimizer.step()
        
        acc_dict[epoch+1] = central_accuracy
        print(f'Epoch {epoch+1} - Global Test Loss: {central_loss:.4f}, Global Test Accuracy: {central_accuracy:.4f}')
    return acc_dict

In [ ]:
training_params = {"epochs": 10, "lr": 0.01, "batch_size": 32, "num_clients": 5}
model_params = {"input_dim": 3072, "hidden_dim": 256, "output_dim": 10}
corrupt_list = [0.95, 0.9375, 0.925, 0.9125, 0.90, 0.875, 0.85, 0.80, 0.65, 0.50, 0.35, 0.20, 0.05]

for corrupt_par in corrupt_list:
    results = FedAvg(1, "CIFAR10", model_params, training_params, corrupt=True, cp=corrupt_par)

    with open(f'NN_CIFAR10_10_C5_{corrupt_par}_Labels', 'wb') as handle:
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)